# Setup

In [15]:
import numpy as np
#%matplotlib widget
from matplotlib import pyplot as plt
import os, time
import pyvisa
from tpo_tools.analysis import CollectionTemplateWF, PUNDTemplateWF, SampleInfo
from tpo_tools.instrumentation import AWGSettings, AWGChannelSettings, OscilloscopeChannelSettings, OscilloscopeSettings, RigolDHO1074, RigolDG2102
from tpo_tools.analysis import SineTemplateWF, ConstantTemplateWF, TrapezoidTemplateWF, DUTInfo
from tpo_tools.analysis import TemplatedTransportData, TransportDataList, PUND
from tpo_tools import fig2clipboard, figures2gif
outdir = './2025-08-05_big_cap/'

try:
    os.mkdir(outdir)
except:
    pass

# Set up AWG/Oscope

In [16]:
#Connect to devices
oscope = RigolDHO1074.identify('TCPIP0::10.97.108.250::INSTR')
# class method, 继承自_BaseParent -> OscilloscopeSettings -> RigolDHO1074
# 在identify中，会调用RigolDHO1074的_idn方法，如果返回True，则返回RigolDHO1074的实例
# 如果返回False，则继续向上查找，直到找到一个能够识别的设备
# 如果都没有找到，则返回None

oscope.verbose= False
oscope.delay = 0.02
awg = RigolDG2102.identify('TCPIP0::10.97.108.216::INSTR')
awg.verbose= False
awg.delay = 0.25

awg.reset()
oscope.reset()
state_set_counter = 0

# awg.verbose = True
# oscope.verbose = True


At address TCPIP0::10.97.108.250::INSTR, RigolDHO1074 query result: RIGOL TECHNOLOGIES,DHO1074,HDO1B244902222,00.02.13
At address TCPIP0::10.97.108.216::INSTR, RigolDG2102 query result: Rigol Technologies,DG2102,DG2P233800951,00.02.07.00.01


AWG and oscilloscope settings. 

In [ ]:
sample_info = SampleInfo(id='Large_Cap_R2C3', notes='NLS_UVA_SAMPLE')
DUTInfo(name='FeCap', area=300**2/4*np.pi, thickness=20e-9, parent=sample_info) # 300 um diameter, 20 nm thick
DUTInfo(name='Heater', area=1e-14, thickness=0, parent=sample_info)


def reset(minium_period = 1e-6, ch1=True, ch2=True):
    awg_settings = AWGSettings()
    oscope_settings = OscilloscopeSettings(sample_rate=2e6, trig_delay=1e-3)

    if ch1:
        awgch1 = AWGChannelSettings(channel=1, sample_rate=1/minium_period, destination='FeCap',  parent=awg_settings)
        OscilloscopeChannelSettings(channel=1, vrange=10, source='FeCap',  parent=oscope_settings)
        OscilloscopeChannelSettings(channel=2, vrange=10, transimpedance=-1e3, source='FeCap',  parent=oscope_settings)
        # amplifier is a inverted  current amp, -1mA goes to 1V
        
        CollectionTemplateWF(parent=awgch1, destination='FeCap')
    else:
        raise ValueError('Always watching the FeCap...')
    
    # ch2 is for the heater, not using in the NLS
    if ch2:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='Heater', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, vrange=5,  source='Heater', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, vrange=5,  transimpedance=+1e4, source='Heater', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='Heater')
    else:
        awgch2 = AWGChannelSettings(channel=2, sample_rate=1e5, destination='none', parent=awg_settings)
        OscilloscopeChannelSettings(channel=3, source='none', parent=oscope_settings)
        OscilloscopeChannelSettings(channel=4, source='none', parent=oscope_settings)
        CollectionTemplateWF(parent=awgch2, destination='none')
    
    return awg_settings, oscope_settings


In [18]:
def set_state(awg_settings, oscope_settings):
    for _ in range(1): # need to run through this twice after first turning on AWG
        # Update awg settings first
        try:
            assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
        except:
            try:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'
            except:
                assert awg.set_state(awg_settings), 'Something went wrong while settings up AWG.'

        # Then update oscope settings
        try:
            assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
        except:
            try:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'
            except:
                assert oscope.set_state( oscope_settings, awg_settings, sample_info ), 'Something went wrong while setting up oscilloscope'

        if oscope_settings.sample_rate != 2e6:
            print('Oscope sample rate adjusted to be', oscope_settings.sample_rate)

        time.sleep(0.2)

# Custom waveforms

In [19]:
cycle_voltage = 5

def set_cycle(NDPU=False):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template
    oscope_settings.sample_rate = 5e5

    polarity = -1 if NDPU else +1 

    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=1e-3, n_cycles=1e2, parent=template1)
    # default n_pulses = 2
    # if n_pulses = 1, it goes like PNPNPNPN---, no delay_time between adjacent pulses
    # using pre-existing PUNDTemplateWF

    set_state(awg_settings, oscope_settings)
    print('~~~~~~~~~~ Set state for PUND~~~~~~~~~~~~')

def set_ND(NDPU=True):
    awg_settings, oscope_settings = reset(ch2=False)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template

    polarity = -1 if NDPU else +1

    PUNDTemplateWF(amplitude=cycle_voltage*polarity, dVdt=1e4, delay_time=5e-3, n_cycles=0.5, n_pulses=2, parent=template1)

    set_state(awg_settings, oscope_settings)
    global state
    state = f'holding_voltage_{voltage_height_rect:.2f}_hold_time_{hold_time_rect:.2f}'
    print('~~~~~~~~~~ Set state for ND ~~~~~~~~~~~~')

def set_rectangular_pulse(voltage_height, hold_time, NDPU=False):
    rise_time = int(hold_time*1e-4)
    if rise_time < 2e-8:
        rise_time = 2e-8
    # default rise time is 20 ns
    awg_settings, oscope_settings = reset(ch2=False, minium_period=rise_time)
    template1, = awg_settings[0].template
    template2, = awg_settings[1].template

    polarity = -1 if NDPU else +1 
    TrapezoidTemplateWF(height = voltage_height, rise_time = rise_time, hold_time = hold_time, offset = 0, parent = template1)
    print(template1)

    set_state(awg_settings, oscope_settings)
    
    global state
    global voltage_height_rect
    global hold_time_rect
    voltage_height_rect = voltage_height
    hold_time_rect = hold_time
    state = f'rectangular_pulse_voltage_{voltage_height:.3f}_hold_time_{hold_time:9f}' if NDPU else f'NP rectangular pulse voltage {voltage_height:.2f} hold time {hold_time:.2f} PU'
    print('~~~~~~~~~~ Set state for', state, '~~~~~~~~~~~~')
    # using rm to open the amplifier after the rectangular pulse



state = ''
print(state)

In [20]:
def trigger(save = True, read = True):
    

    if not read:
        try:
            with rm.open_resource('ASRL7::INSTR') as tia:
                tia.write(':INPUT ON')
                print(tia.query(':INPUT?'))

        except:
            print('TIA is not connected')
    else:
        try:
            with rm.open_resource('ASRL7::INSTR') as tia:
                tia.write(':INPUT OFF')
                print(tia.query(':INPUT?'))
        except:
            print('Something Went Wrong')

    oscope.prepare_for_trigger()
    awg.prepare_for_trigger()
    oscope.trigger()
    # oscope是01074，awg是33522B
    # 中间有mdep/srate的delay time，保证awg的波形被完全capture

    if read:
        data = oscope.read_data()
        data.sample_info = sample_info 
    

    awg.done_with_trigger() # turn off outputs, since oscope在按下tforce之后会一直acquaire波形， so only solution is to turn off awg outputs
    oscope.done_with_trigger() 

    
    print('Data read') 

    if not save:
        return
    
    data.save(f'{outdir}/{data[0].daq_timeofday}_{state}.hdf5') 
    return data
#trigger(save=False)

## Take/Save Data

In [25]:
oscope.opc = True


pund_data = []
rm = pyvisa.ResourceManager()
import time

# Main loop to collect data

set_cycle(NDPU=False)
t0 = time.time()
trigger(save = False)
print('debug 1', time.time()-t0)

set_rectangular_pulse(voltage_height = 2, hold_time = 100,  NDPU=True)
print('debug 2', time.time()-t0)

trigger(save = False, read = False)
print('debug 3', time.time()-t0)

set_ND(NDPU=True)
pund_data.append(trigger())


print('Done!')

Deleting TemplatedTransportData
Offset	 0.499 	 4.990000E-1
Memdep	 1000000.0 	 1.000E+06
Secdiv	 0.1 	 1.000000E-1
Oscope sample rate adjusted to be 500000.0
~~~~~~~~~~ Set state for PUND~~~~~~~~~~~~
0

Data read
Deleting TemplatedTransportData
debug 1 7.566778898239136
Offset	 99.8 	 9.980000E+1
Memdep	 25000000.0 	 2.500E+07
Secdiv	 20.0 	 2.000000E+1
~~~~~~~~~~ Set state for rectangular_pulse_voltage_2.000_hold_time_100.000000 ~~~~~~~~~~~~
debug 2 19.778403759002686
1

Data read
debug 3 223.8282437324524
Offset	 0.009000000000000001 	 9.000000E-3
Memdep	 100000.0 	 1.000E+05
Secdiv	 0.002 	 2.000000E-3
~~~~~~~~~~ Set state for ND ~~~~~~~~~~~~
0

Data read
Done!
